In [1]:
!export LC_ALL=en_US.UTF-8
import json
from pprint import pprint
import re
import numpy as np
from embedding import *

In [2]:
import tensorflow as tf

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [5]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def find_sub_list_2(sl ,l):
    start = 0
    end = len(sl)
    while end < len(l):
        if l[start:end] == sl:
            return (start,right)
        start,end = start+1,end+1
    return (-1,-1)
    

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [6]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [7]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [8]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [9]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

In [10]:
reduced_glove = np.array(reduced_glove)

In [11]:
reduced_glove.shape

(50000, 300)

In [12]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1510
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [13]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [14]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [15]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1510
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [16]:
examples_to_take_train = 100

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [17]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [42]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [47]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i,j] = look_up_word_reduced(word)

(100, 459)


In [48]:
print(X_train_ans[13])
print(Y_train_ques[13])
print(suppression_answer[13])

['learn', 'and', 'practice', 'that', 'trade', 'from', 'a', 'very', 'early', 'age']
['for', 'working', 'parents', 'what', 'is', 'the', 'value', 'of', 'having', 'their', 'children', 'work', 'by', 'their', 'side', '?']
[3425    7  811   19  438   23   10  256  105  302    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [20]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [21]:
question_input_tokens[0]

array([   1,   26,  135, 4787,  682,  135,   58, 1625,  896,   19,   66,
        856,    9, 5936,   16,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [22]:
reduced_glove.shape

(50000, 300)

In [23]:
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

4194
244
4064


In [24]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [25]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [26]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [27]:
document_tokens[0]

array([   10,   206,   670,     5,   896,  1793,    15,  1072,    32,
        2269,    14,    11,    10,  1114,     5,    42,  9478,    20,
          46,  1667,   896,     4,  1279,   365,    15,    32,  1279,
         106,  1843,     4,   304,    32,    49,    14,     4,    32,
        1664,  4338,    19,  3643,  1482,    23,     3,   362,    15,
          54,    13,    10,  2913,    32, 18085,  1667,  1336,     9,
          34,   749,    22,     3,  4044,   197,   119,   585,     9,
         240,    10,  2174,  1667,     4,    28,   456,  9589,     8,
          10,  2174,  5997,    14,     6,    17,   196,     4,     8,
           3,   468,     5,    10,  1114,     5, 10205,  9478,     4,
          36,    66,  1245,   145,     3, 13741,  1300,     7,  1338,
         896,    81,    22,     3,  4854,   826,     5,     3,  1664,
        1364,     4,    28,    41,    65,  1553,    79,    34,  1452,
           6,    36,  1247,     8,     3, 13779,  3509,    19,     3,
        4854,   188,

In [28]:
answer_labels.shape

(100, 459)

In [72]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]


d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


In [73]:
a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

In [74]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)

_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

In [75]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

#helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths2, name="helper")
helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)


decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")


Tensor("question_loss/truediv:0", shape=(), dtype=float32)


Tensor("Shape_7:0", shape=(2,), dtype=int32)


In [80]:
#Suppression Loss
s_answer = tf.placeholder(tf.int64, shape=[None,None], name="suppression_answer")
suppression_loss = 0
lambdaSuppress = 0.5

num_classes = array_ops.shape(decoder_outputs)[2]
decoder_outputs_flat = tf.reshape(decoder_outputs, [-1, num_classes])
s_answer_flat = tf.reshape(s_answer, [-1])

print(decoder_outputs_flat)
print(s_answer_flat)


for batchNum in tf.range(tf.shape(s_answer)[0]):
    for output in decoder_labels[batchNum]:
        for sup in s_answer[batchNum][0:encoder_lengths[batchNum]]:
            suppression_loss = suppression_loss + decoder_outputs[batchNum][output][sup]
suppression_loss = suppression_loss * lambdaSuppress






print(suppression_loss)

#Maximize Entropy Loss



Tensor("Reshape_2:0", shape=(?, ?), dtype=float32)
Tensor("Reshape_3:0", shape=(?,), dtype=int64)


TypeError: 'Tensor' object is not iterable.

In [70]:
tf.gather()

array([1, 0, 1, 1, 0, 1, 0, 1, 1, 1])

In [ ]:
loss = tf.add(answer_loss, question_loss, name="loss")
loss = tf.add(loss, suppression_loss, name="loss")

In [ ]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [ ]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [33]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
            
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [35]:
batch_size = 25
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)

In [36]:
len(batch_input[0])

4

In [39]:
saved_vars = []
l = len(tf.all_variables())
for i,var in enumerate(tf.all_variables()):
    print(i,"/",l)
    saved_vars.append(var)
        
print(len(saved_vars))

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Please use tf.global_variables instead.
0 / 20
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
20


In [40]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

#saver = tf.train.Saver()
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#session = tf.Session(config=config)

session.run(tf.global_variables_initializer())
session.run(tf.variables_initializer(saved_vars))


EPOCHS = 400

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        _, loss_value = session.run([optimizer, loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
        })
        print("Loss: {0}".format(loss_value))
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")


Epoch 1
Batch :  0
Loss: 11.485234260559082
Batch :  1
Loss: 11.43755054473877
Batch :  2
Loss: 10.013688087463379
Batch :  3
Loss: 8.474026679992676
Epoch 2
Batch :  0
Loss: 5.911814212799072
Batch :  1


KeyboardInterrupt: 

In [233]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

'qgen-model'

In [41]:
batch_input[3][0].shape

(25, 14, 263)

In [37]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

INFO:tensorflow:Restoring parameters from qgen-model


In [82]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][48],
    d_lengths: batch_input[1][48],
})
answers = np.argmax(answers, 2)

In [92]:
for i in range(28):
    print("Prediction")
    printAllAns(answers,48,0)
    print("Ground Truth")
    printAllAns(batch_input[2][48],48,0)

Prediction
mendes revealed seven months . mendes also rome kodak mm london , craig and  
Ground Truth
pinewood studios , seven months . mendes london , mexico city and rome . kodak 35 mm film stock .  
Prediction
mendes revealed seven months . mendes also rome kodak mm london , craig and  
Ground Truth
pinewood studios , seven months . mendes london , mexico city and rome . kodak 35 mm film stock .  
Prediction
mendes revealed seven months . mendes also rome kodak mm london , craig and  
Ground Truth
pinewood studios , seven months . mendes london , mexico city and rome . kodak 35 mm film stock .  
Prediction
mendes revealed seven months . mendes also rome kodak mm london , craig and  
Ground Truth
pinewood studios , seven months . mendes london , mexico city and rome . kodak 35 mm film stock .  
Prediction
mendes revealed seven months . mendes also rome kodak mm london , craig and  
Ground Truth
pinewood studios , seven months . mendes london , mexico city and rome . kodak 35 mm film 

In [85]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    
printQues(47,0)
    

<START> which bond novel first featured spectre ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  


In [176]:
import itertools

batchNum = 80

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [177]:
batch_input[0][batchNum].shape

(64, 451)

In [178]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [179]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

---------------------------------------------------------------------------------------------
Generated Question: what son of the umayyads were used to break up both the roman catholic church ?
Ground Truth Question: 
<START> what son of <UNK> was the ancestor of the umayyads ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['umayya']
Context:
according to tradition , the umayyad family ( also known as the banu <UNK> ) and muhammad both descended from a common ancestor , abd manaf ibn qusai , and they originally came from the city of mecca . muhammad descended from abd <UNK> via his son hashim , while the umayyads descended from abd manaf via a different son , <UNK> , whose son was umayya . the two families are therefore considered to be different clans ( those of hashim and of umayya , respectively ) of the same tribe ( that of the <UNK> ) . however muslim shia historians suspect that umayya was an adopted son of abd shams so he was not a blood relative 

with constantinople as its capital and control of lands around the mediterranean basin , the ottoman empire was at the centre of interactions between the eastern and western worlds for six centuries . following a long period of military setbacks against european powers , the ottoman empire gradually declined into the late nineteenth century . the empire allied with germany in the early 20th century , with the imperial ambition of recovering its lost territories , joining in world war i to achieve this ambition on the side of germany and the central powers . while the empire was able to largely hold its own during the conflict , it was struggling with internal dissent , especially with the arab revolt in its arabian holdings . starting before the war , but growing increasingly common and violent during it , major atrocities were committed by the ottoman government against the armenians , assyrians and pontic greeks . the empire 's defeat and the occupation of part of its territory by th

located in the south atlantic ocean on the mid-atlantic ridge , more than 2,000 kilometres ( 1,200 mi ) from the nearest major landmass , saint helena is one of the most remote places in the world . the nearest port on the continent is <UNK> in southern angola , and the nearest international airport the quatro de <UNK> airport of angola 's capital luanda ; connections to cape town in south africa are used for most shipping needs , such as the mail boat that serves the island , the rms st helena . the island is associated with two other isolated islands in the southern atlantic , also british territories : ascension island about 1,300 kilometres ( 810 mi ) due northwest in more equatorial waters and tristan da cunha , which is well outside the tropics 2,430 kilometres ( 1,510 mi ) to the south . the island is situated in the western hemisphere and has the same longitude as cornwall in the united kingdom . despite its remote location , it is classified as being in west africa by the unit

in its first century and half , the eic used a few hundred soldiers as guards . the great expansion came after 1750 , when it had 3000 regular troops . by 1763 , it had 26,000 ; by 1778 , it had <UNK> it recruited largely indian troops , and trained them along european lines . the company , fresh from a colossal victory , and with the backing of its own private <UNK> and experienced army , was able to assert its interests in the carnatic region from its base at madras and in bengal from calcutta , without facing any further obstacles from other colonial powers . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>

<START> who was added to the tour in season 12 ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['a', 'semi-finalist', 'who', 'won', 'a', 'sing-off']
Context:
the top ten ( eleven in season ten ) toured at the end of every season . in the season twelve tour a semi-finalist who won a sing-off was also added to the tour . kellogg 's <UNK> was the sponsor for the first seven seasons , and guitar hero was added for the season seven tour . m & m 's pretzel chocolate candies was a sponsor of the season nine tour . the season five tour was the most successful tour with gross of over $ 35 million . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>

...

with the treaty of the pyrenees ( 1659 ) , spain ceded the northern part of catalonia to france , and soon thereafter the local catalan varieties came under the influence of french , which in 1700 became the sole official language of the region . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <EN

on august 6 , 2010 , the cbc issued a press release stating that due to financial reasons , the cbc and radio-canada would only transition 27 transmitters total , one in each market where there was an originating station ( i.e . a cbc or radio-canada television station located in that market ) . further , the cbc stated in the release , that only 15 of the transmitters would be in place by august 31 , 2011 due to lack of available funds , and that the remainder would not be on the air until as late as august 31 , <UNK> additionally , the cbc stated in the release that it was asking the crtc for permission to continue broadcasting in analogue until the identified transmitters for transition were up and running . at the time of the press release , only eight of the corporation 's transmitters ( four cbc and four radio canada ) were broadcasting in digital . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> 

after years of negotiation , three great powers , russia , the united kingdom and france , decided to intervene in the conflict and each nation sent a navy to greece . following news that combined <UNK> fleets were going to attack the greek island of hydra , the allied fleet intercepted the <UNK> fleet at navarino . after a week-long standoff , a battle began which resulted in the destruction of the <UNK> fleet . a french expeditionary force was dispatched to supervise the evacuation of the egyptian army from the peloponnese , while the greeks proceeded to the captured part of central greece by <UNK> as a result of years of negotiation , the nascent greek state was finally recognized under the london protocol in 1830 . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <

greece played a peripheral role in the war . when russia attacked the ottoman empire in 1853 , king otto of greece saw an opportunity to expand north and south into ottoman areas that had large greek christian majorities . however , greece did not coordinate its plans with russia , did not declare war , and received no outside military or financial support . greece , an orthodox nation , had considerable support in russia , but the russian government decided it was too dangerous to help greece expand its holdings . <UNK> when the russians invaded the principalities , the ottoman forces were tied down so greece invaded thessaly and epirus . to block further greek moves , the british and french occupied the main greek port at piraeus from april 1854 to february 1857 , and effectively neutralized the greek army . greeks , gambling on a russian victory , incited the large-scale epirus revolt of 1854 as well as uprisings in crete . the insurrections were failures that were easily crushed by

eritrea ( <UNK> or <UNK> ; , officially the state of eritrea , is a country in east africa . with its capital at asmara , it is bordered by sudan in the west , ethiopia in the south , and djibouti in the southeast . the northeastern and eastern parts of eritrea have an extensive coastline along the red sea . the nation has a total area of approximately <UNK> km2 ( <UNK> sq mi ) , and includes the dahlak archipelago and several of the hanish islands . its name eritrea is based on the greek name for the red sea ( <UNK> <UNK> <UNK> thalassa ) , which was first adopted for italian eritrea in 1890 . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <E

In [132]:
batch_input[5][18].shape

(32, 20)

In [80]:
batch_input[5][1].shape

(30, 19)

In [81]:
batch_input[6][1].shape

(30, 19)

In [62]:
max(batch_input[7][1])

19

20

In [56]:
max_question_len

20

In [130]:
len(batch_input)

9